In [ ]:
import os
import numpy as np
from PIL import Image

# Veri seti yolu
data_path = "/content/drive/MyDrive/malaria/cell_images/cell_images"

# Sınıf etiketleri
class_names = ["Parasitized", "Uninfected"]

# Veri setindeki görüntü sayısı
num_samples = sum([len(files) for r, d, files in os.walk(data_path)])

# Veri seti boyutları
img_height, img_width = 64, 64
num_channels =3

# X ve y için numpy dizileri
X = np.zeros((num_samples, img_height, img_width, num_channels), dtype=np.float32)
y = np.zeros((num_samples,), dtype=np.int32)

# Verileri yükleyip numpy dizilerine dönüştürme
index = 0
for i, class_name in enumerate(class_names):
    class_path = os.path.join(data_path, class_name)
    for img_name in os.listdir(class_path):
        img_path = os.path.join(class_path, img_name)
        img = Image.open(img_path)
        img = img.resize((img_height, img_width))
        X[index] = np.array(img, dtype=np.float32) / 255.
        y[index] = i
        index += 1

# Veri setini karıştırma
np.random.seed(42)
shuffle_index = np.random.permutation(num_samples)
X, y = X[shuffle_index], y[shuffle_index]

# Veri setini train, test ve validation olarak ayırma
num_train_samples = int(0.7 * num_samples)
num_test_samples = int(0.2 * num_samples)
num_val_samples = num_samples - num_train_samples - num_test_samples

x_train, y_train = X[:num_train_samples], y[:num_train_samples]
x_test, y_test = X[num_train_samples:num_train_samples+num_test_samples], y[num_train_samples:num_train_samples+num_test_samples]
x_val, y_val = X[num_train_samples+num_test_samples:], y[num_train_samples+num_test_samples:]

# Dizileri npy dosyalarına kaydetme
np.save('x_train.npy', x_train)
np.save('y_train.npy', y_train)
np.save('x_test.npy', x_test)
np.save('y_test.npy', y_test)
np.save('x_val.npy', x_val)
np.save('y_val.npy', y_val)


In [ ]:
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.utils import to_categorical

In [ ]:
# Dizileri yükleme
x_train = np.load('x_train.npy')
y_train = np.load('y_train.npy')
x_test = np.load('x_test.npy')
y_test = np.load('y_test.npy')
x_val = np.load('x_val.npy')
y_val = np.load('y_val.npy')

In [ ]:
# CNN Modelini Oluşturma
cnn_model = Sequential()
cnn_model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, num_channels)))
cnn_model.add(MaxPooling2D(pool_size=(2, 2)))
cnn_model.add(Conv2D(64, (3, 3), activation='relu'))
cnn_model.add(MaxPooling2D(pool_size=(2, 2)))
cnn_model.add(Flatten())
cnn_model.add(Dense(128, activation='relu'))
cnn_model.add(Dense(1, activation='sigmoid'))

In [ ]:
# CNN Modelini eğitme
cnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
cnn_model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_val, y_val))

Epoch 1/10
603/603 [==============================] - 16s 8ms/step - loss: 0.4581 - accuracy: 0.7746 - val_loss: 0.2379 - val_accuracy: 0.9090
Epoch 2/10
603/603 [==============================] - 4s 6ms/step - loss: 0.1978 - accuracy: 0.9267 - val_loss: 0.1927 - val_accuracy: 0.9333
Epoch 3/10
603/603 [==============================] - 4s 6ms/step - loss: 0.1680 - accuracy: 0.9434 - val_loss: 0.1664 - val_accuracy: 0.9423
Epoch 4/10
603/603 [==============================] - 5s 8ms/step - loss: 0.1405 - accuracy: 0.9529 - val_loss: 0.1725 - val_accuracy: 0.9416
Epoch 5/10
603/603 [==============================] - 4s 6ms/step - loss: 0.1169 - accuracy: 0.9610 - val_loss: 0.1727 - val_accuracy: 0.9398
Epoch 6/10
603/603 [==============================] - 4s 6ms/step - loss: 0.0995 - accuracy: 0.9653 - val_loss: 0.1570 - val_accuracy: 0.9496
Epoch 7/10
603/603 [==============================] - 4s 7ms/step - loss: 0.0782 - accuracy: 0.9720 - val_loss: 0.1673 - val_accuracy: 0.9456
Epoch

In [ ]:
# CNN modelinden özellik çıkarma
train_features = cnn_model.predict(x_train)
test_features = cnn_model.predict(x_test)

173/173 [==============================] - 1s 3ms/step


In [ ]:
# SVM Modelini Oluşturma ve eğitme
svm_model = SVC()
svm_model.fit(train_features, y_train)


SVC()

In [ ]:
# Hibrit modelin performansını değerlendirme
train_predictions = svm_model.predict(train_features)
test_predictions = svm_model.predict(test_features)

train_accuracy = accuracy_score(y_train, train_predictions)
test_accuracy = accuracy_score(y_test, test_predictions)

print("Eğitim verisi doğruluğu:", train_accuracy)
print("Test verisi doğruluğu:", test_accuracy)


Eğitim verisi doğruluğu: 0.995334370139969
Test verisi doğruluğu: 0.9468336055162403
